## Title: Koppen-Geiger Climate Changes - 1901 - 2100

### Description
Köppen climate classification is a widely used vegetation-based empirical climate classification system developed by German botanist-climatologist Wladimir Köppen. It's based on the idea that climate is best defined by native vegetation. This particular version of Köppen-Geiger shows the changes in the climate regions and biomes over time (from 1901 - 2100) using observations for 1901 - 2000 and the A1F1 IPCC climate scenario for 2001 - 2100. A1F1 is the worst case scenario future projection for climate change modeling. You can find out more information on how this was made by visiting World Maps of Köppen-Geiger Climate Classisification website.

### FLINT
This dataset has been pre-processed/checked and is suitable for use in FLINT. Please adhere to individual dataset licence conditions and citations. Processed data can be accessed here: https://datasets.mojaglobal.workers.dev/ <br>

### Format
<b>Extent: </b>Global coverage<br>
<b>Format</b>: polygon geoJSON .json<br>
<b>Cordinate system:</b> EPSG:4326 (WGS84)<br>
<b>Temporal Resolution: </b>1901-2100<br>
<b>Size:</b> 40 MB <br>

### Original source
Original Source: http://koeppen-geiger.vu-wien.ac.at/shifts.htm or https://sos.noaa.gov/datasets/koppen-geiger-climate-changes-1901-2100/ Accessed 13/12/2020 <br>
Viewer: https://www.arcgis.com/home/item.html?id=7a53584fa55643df969f93cec83788e1

### Licence
Only non-commercial scientific or educational use. Please attribute using citation.

### Citation
Rubel, F., and M. Kottek, 2010: Observed and projected climate shifts 1901-2100 depicted by world maps of the Köppen-Geiger climate classification. Meteorol. Z. , 19, 135-141. DOI: 10.1127/0941-2948/2010/0430. http://koeppen-geiger.vu-wien.ac.at/pdf/Paper_2010.pdf

### Original format
Global coverage, vector, shapefile<br>
Cordinate system: Bassel

### Metadata
http://koeppen-geiger.vu-wien.ac.at/shifts.htm or https://sos.noaa.gov/datasets/koppen-geiger-climate-changes-1901-2100/ <br>
Climate scenarios available <br>
Observed:    1901-1925    1926-1950    1951-1975    1976-2000<br>
A1FI:    2001-2025    2026-2050    2051-2075    2076-2100<br>
A2:    2001-2025    2026-2050    2051-2075    2076-2100<br>
B1:    2001-2025    2026-2050    2051-2075    2076-2100<br>
B2:    2001-2025    2026-2050    2051-2075    2076-2100<br>
Each contains vector data for the respective period and emission scenario. Use the scheme below to convert the integers to climate classes:<br>
11 ... Af<br>
12 ... Am<br>
13 ... As<br>
14 ... Aw<br>
21 ... BWk<br>
22 ... BWh<br>
26 ... BSk<br>
27 ... BSh<br>
31 ... Cfa<br>
32 ... Cfb<br>
33 ... Cfc<br>
34 ... Csa<br>
35 ... Csb<br>
36 ... Csc<br>
37 ... Cwa<br>
38 ... Cwb<br>
39 ... Cwc<br>
41 ... Dfa<br>
42 ... Dfb<br>
43 ... Dfc<br>
44 ... Dfd<br>
45 ... Dsa<br>
46 ... Dsb<br>
47 ... Dsc<br>
48 ... Dsd<br>
49 ... Dwa<br>
50 ... Dwb<br>
51 ... Dwc<br>
52 ... Dwd<br>
61 ... EF<br>
62 ... ET<br>

### Notes
The original layers contain topological errors (non-simple geometry) - and an overlapping region at the international time/date line in three of the climate models that are fixed in the json using the below code. Also note, some scenario datasets are simplified polygon type converted from grid, others are direct polygon conversion from grid - as per original downloaded 19/12/2020. Gaps in data are water bodies and are valid.

### Processing
Repair geometry, fix topologial error (remove overlaps), convert to geojson, EPSG:4326 (WGS84), remove/disable Z values.

In [ ]:
import arcpy
import os

#folder of feature classes to process
in_folder = r"C:\data\KoeppenGeiger\all"
scr_folder = r"C:\data\scratch.gdb"
out_folder = r"C:\data\json\KoeppenGeiger"

# Set environments
workspace = in_folder
arcpy.env.workspace = workspace
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4326)
arcpy.env.outputZFlag = "Disabled"
arcpy.env.overwriteOutput = True

# Execute CreateFileGDB
if os.path.exists(r"C:\data\scratch.gdb"):
    arcpy.Delete_management(r"C:\data\scratch.gdb")
    scr = arcpy.CreateFileGDB_management(r"C:\data", "scratch")

featureclasses = arcpy.ListFeatureClasses()
print(featureclasses)

# Iterate through all datasets, repair and convert to json
for fc in featureclasses:
    fcname = os.path.join(os.path.splitext(fc)[0])
    outjson = os.path.join(out_folder, fcname)
    
    # Process: Make Feature Layer
    fLayer = "project_Layer"
    arcpy.management.MakeFeatureLayer(fc, fLayer)

    # Process: Union - fill gaps and find overlaps
    projectIntersect = os.path.join(scr_folder, "projectIntersect")
    arcpy.analysis.Intersect(fLayer, projectIntersect, "ONLY_FID")

    # Process: Multipart To Singlepart - explode into single parts
    projectSingle = os.path.join(scr_folder, "projectSingle")
    arcpy.management.MultipartToSinglepart(projectIntersect, projectSingle)

    # Process: Select - select overlaps and slithers smaller than 0.00001 degrees (this may need to be checked)
    intSelect = os.path.join(scr_folder, "intSelect")
    arcpy.analysis.Select(projectSingle, intSelect)

    # Process: Dissolve - dissolve slithers and overlaps to single features
    dissolveSlither = os.path.join(scr_folder, "dissolveSlither")
    arcpy.management.Dissolve(intSelect, dissolveSlither, None, None,"SINGLE_PART")

    # Process: Erase overlaps
    removeSlither = os.path.join(scr_folder, "removeSlither")
    arcpy.analysis.Erase(fLayer, dissolveSlither, removeSlither)

    # Process: Append small slithers and gap fills to be merged into coincident polygons with largest boundary
    appendSlither = arcpy.management.Append(dissolveSlither, removeSlither, "NO_TEST")

    # Process: Eliminate slither by merge into coincident polygons with largest boundary
    eliminateSlither = os.path.join(scr_folder, "eliminateSlither")
    selectSlither = arcpy.management.SelectLayerByAttribute(appendSlither, "NEW_SELECTION", "OBJECTID IS NULL")
    arcpy.management.Eliminate(selectSlither, eliminateSlither, "LENGTH")
    print(fcname, ' repairing...')

    # Process: Repair Geometry (non-simple geometry)
    geomRepair = arcpy.management.RepairGeometry(eliminateSlither, "DELETE_NULL", "OGC")[0]

    # Process: Features To JSON
    arcpy.conversion.FeaturesToJSON(eliminateSlither, outjson, "NOT_FORMATTED", "NO_Z_VALUES", "NO_M_VALUES", "GEOJSON", "WGS84", "USE_FIELD_NAME")
    print(outjson, '.geojson complete')
    

arcpy.AddMessage("All done!")
print('done')